<a href="https://colab.research.google.com/github/avkaz/DeepLearningPetIdentification/blob/preprocess_pipeline/preprop_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import json
from PIL import Image, ExifTags, ImageDraw
import requests
import io
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import itertools
from itertools import islice

# Metadata Fetching
def get_data():
    """
    Fetches and parses JSON data from the given URL.

    Returns:
        dict: The parsed JSON data as a Python dictionary.
    """
    url = "https://raw.githubusercontent.com/avkaz/DeepLearningPetIdentification/main/pets_db.json"

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data
    except requests.RequestException as e:
        print(f"An error occurred while fetching data: {e}")
        raise
    except json.JSONDecodeError as e:
        print(f"An error occurred while parsing JSON: {e}")
        raise

# Model Loading
detector = None
MODEL_URL = "https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_320x320/1"





Total number of "pets", where metadata: "images" is empty.

In [31]:
def load_detector_model():
    global detector
    if detector is None:
        print("Uploading model...")
        detector = hub.load(MODEL_URL).signatures['serving_default']
        print("Model loaded successfully.")
    else:
        pass

In [32]:
def filter_metadata_with_images(metadata):
    """
    Filters metadata to include only entries with non-empty 'images' lists.

    Args:
        metadata (dict): The original metadata dictionary.

    Returns:
        dict: A filtered metadata dictionary with entries that have images.
    """
    return {key: value for key, value in metadata.items() if value.get("images")}

In [33]:
# Metadata Verification
def verify_metadata(metadata):
    """
    Verifies metadata integrity by checking for missing or inconsistent entries.

    Args:
        metadata (dict): The metadata dictionary to verify.
    """
    for key, value in metadata.items():
        if not isinstance(value, dict) or "Plemeno" not in value or "Barva" not in value or "Věk" not in value or "Velikost" not in value or "images" not in value:
            print(f"Warning: Incomplete metadata for key {key}: {value}")

# Function to fix orientation using EXIF
def fix_orientation(image):
    """
    Adjust the image orientation based on its EXIF metadata to account for camera rotation.
    The function looks for the 'Orientation' tag in the EXIF data and rotates the image accordingly.

    Arguments:
    image -- The image to fix the orientation for (PIL Image object).

    Returns:
    PIL Image with corrected orientation.
    """
    try:
        for orientation in ExifTags.TAGS.keys():
            if ExifTags.TAGS[orientation] == 'Orientation':
                break
        exif = image._getexif()
        if exif is not None:
            orientation = exif.get(orientation)
            if orientation == 3:
                image = image.rotate(180, expand=True)
            elif orientation == 6:
                image = image.rotate(270, expand=True)
            elif orientation == 8:
                image = image.rotate(90, expand=True)
    except (AttributeError, KeyError, IndexError):
        pass
    return image

# Function to crop and resize the image based on a bounding box

    """
    Crops the image using a given bounding box and then resizes it to the target size.

    Arguments:
    image -- The image to crop and resize (TensorFlow Tensor).
    bounding_box -- A tuple (x1, y1, x2, y2) specifying the coordinates of the bounding box.
    target_size -- The target size (height, width) to resize the image to.

    Returns:
    The cropped and resized image (TensorFlow Tensor).
    """

def crop_and_resize(image, bounding_box, target_size):
    image = tf.convert_to_tensor(image, dtype=tf.float32)
    x1, y1, x2, y2 = bounding_box
    image = tf.strided_slice(image, [int(y1), int(x1), 0], [int(y2), int(x2), 3])
    image = tf.image.resize(image, target_size)
    return image

# Function to detect pets in the image (Placeholder function, adjust as needed)
def detect_pet(image):
    load_detector_model()
    input_tensor = tf.image.resize(image, [640, 640]) / 255.0
    input_tensor = tf.expand_dims(input_tensor, axis=0)
    input_tensor_uint8 = tf.cast(input_tensor * 255.0, tf.uint8)

    result = detector(tf.convert_to_tensor(input_tensor_uint8))
    result = {key: value.numpy() for key, value in result.items()}

    if 'detection_classes' in result and 'detection_scores' in result:
        detected_classes = result['detection_classes']
        detected_boxes = result['detection_boxes']
        detected_scores = result['detection_scores']
        pet_classes = [b"Cat", b"Dog", b"Animal"]

        for idx in range(len(detected_classes[0])):
            detected_class = detected_classes[0][idx]
            detected_score = detected_scores[0][idx]
            detected_box = detected_boxes[0][idx]

            if detected_class in pet_classes and detected_score > 0.5:
                return detected_box
    return None

# Function to visualize the image
def visualize_image(image, title="Processed Image", visualize=False):
    """
    Visualizes the processed image using Matplotlib.

    Arguments:
    image -- The image to visualize, can be a TensorFlow tensor or a NumPy array.
    title -- The title to display on top of the image.
    visualize -- A flag to control whether to visualize the image. Default is True.
    """
    if visualize:
        # Convert TensorFlow tensor to NumPy array if necessary
        if isinstance(image, tf.Tensor):
            image = image.numpy()

        # If it's an RGB image, clip pixel values to the range [0, 1]
        if image.ndim == 3 and image.shape[-1] == 3:
            image = np.clip(image, 0, 1)
        elif image.ndim == 2:  # If grayscale, clip to [0, 255]
            image = np.clip(image, 0, 255).astype(np.uint8)

        # Show the image using Matplotlib
        plt.imshow(image)
        plt.title(title)
        plt.axis("off")
        plt.show()

# Download and Preprocess Image
def download_and_preprocess_image(url, target_size=(224, 224), visualize=False):
    response = requests.get(url)
    image_bytes = response.content
    pil_image = Image.open(io.BytesIO(image_bytes))
    pil_image = fix_orientation(pil_image)

    image = tf.convert_to_tensor(np.array(pil_image), dtype=tf.float32) / 255.0
    bounding_box = detect_pet(image)

    if bounding_box is not None:
        image = crop_and_resize(image, bounding_box, target_size)
    else:
        # If no pet detected, resize with padding
        image = tf.image.resize_with_crop_or_pad(image, target_size[0], target_size[1])

    # Visualize the image if needed
    visualize_image(image, title="Processed Image", visualize=visualize)

    return image

This function processes image data from a metadata dictionary and prepares it for training in a machine learning model using TensorFlow.

In [36]:
#def preprocess_pipeline(metadata, target_size=(224, 224), visualize=False):
 #   images = []
  #  labels = []
   # for key, entry in metadata.items():
    #    image_urls = entry["images"]

        # Create a label (list of characteristics)
     #   label = [
      #      entry.get("Plemeno", "Unknown"),
       #     entry.get("Věk", "Unknown"),
        #    entry.get("Barva", "Unknown"),
         #   entry.get("Velikost", "Unknown")

        # Process all the images in the 'images' list
      #  for image_url in image_urls:
       #     image = download_and_preprocess_image(image_url, target_size, visualize)
        #    images.append(image)
         #   labels.append(label)  # Append the same label for each image

  #  return tf.data.Dataset.from_tensor_slices((images, labels))

def preprocess_dataset(metadata, target_size=(224, 224)):
    """
    Preprocesses the dataset to create pairs of images for the same pet and different pets.
    The output dataset contains the attributes and image pairs.
    """
    data_pairs = []

    # First, create image pairs from the same pet (label = 1)
    for key, entry in metadata.items():
        # Get the pet attributes
        plemeno = entry.get("Plemeno", "Unknown")
        vek = entry.get("Věk", "Unknown")
        barva = entry.get("Barva", "Unknown")
        velikost = entry.get("Velikost", "Unknown")

        # Get all the images for this pet
        images = entry.get("images", [])

        #if len(images) < 2:
         #   continue  # Skip pets with fewer than 2 images

        # Pair each image with every other image for the same pet
        for i in range(len(images)):
            for j in range(i + 1, len(images)):
                # Download and preprocess both images
                image1 = download_and_preprocess_image(images[i], target_size)
                image2 = download_and_preprocess_image(images[j], target_size)

                # Add to data pairs with label = 1 (same pet)
                data_pairs.append(((plemeno, vek, barva, velikost, image1, image2), 1))

    # Now create image pairs from different pets (label = 0)
    all_keys = list(metadata.keys())
    for key1, key2 in itertools.combinations(all_keys, 2):
        entry1 = metadata[key1]
        entry2 = metadata[key2]

        # Get the attributes and images for both pets
        plemeno1 = entry1.get("Plemeno", "Unknown")
        vek1 = entry1.get("Věk", "Unknown")
        barva1 = entry1.get("Barva", "Unknown")
        velikost1 = entry1.get("Velikost", "Unknown")
        images1 = entry1.get("images", [])

        plemeno2 = entry2.get("Plemeno", "Unknown")
        vek2 = entry2.get("Věk", "Unknown")
        barva2 = entry2.get("Barva", "Unknown")
        velikost2 = entry2.get("Velikost", "Unknown")
        images2 = entry2.get("images", [])

        if not images1 or not images2:
            continue  # Skip if either pet doesn't have images

        # Pair the first image of pet 1 with the first image of pet 2 (label = 0)
        image1 = download_and_preprocess_image(images1[0], target_size)
        image2 = download_and_preprocess_image(images2[0], target_size)

        # Add to data pairs with label = 0 (different pets)
        data_pairs.append(((plemeno1, vek1, barva1, velikost1, image1, image2), 0))

    # Convert to a TensorFlow dataset
    dataset = tf.data.Dataset.from_tensor_slices(data_pairs)
    return dataset

This function will display a specified number of sample images from the dataset, with the corresponding labels shown as the image title.

In [7]:
# Dataset Sample Visualization
def visualize_dataset_sample(dataset, num_samples=5):
    for image, label in dataset.take(num_samples):
        plt.imshow(image.numpy())
        plt.title(f"Label: {label.numpy().decode()}")
        plt.axis("off")
        plt.show()

Split data into parts: training, validation and test data.

---



In [8]:
def split_data(dataset, val_split=0.1, test_split=0.1):
    total_size = len(dataset)
    val_size = int(val_split * total_size)
    test_size = int(test_split * total_size)

    val_dataset = dataset.take(val_size)
    test_dataset = dataset.skip(val_size).take(test_size)
    train_dataset = dataset.skip(val_size + test_size)

    return train_dataset, val_dataset, test_dataset


This function prepares the dataset by batching it into groups of a specified size. It can also shuffle the dataset before batching and prefetch it to optimize performance.

In [9]:
# Dataset Batching
def batch_dataset(dataset, batch_size=32, shuffle=True):
    if shuffle:
        dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

In [ ]:
if __name__ == "__main__":
    print("Fetching metadata...")
    metadata = get_data()
    verify_metadata(metadata)

    #print("Filtering metadata to remove entries without images...")
    filtered_metadata = filter_metadata_with_images(metadata)
    print(f"Filtered metadata contains {len(filtered_metadata)} entries (original: {len(metadata)})")

    # Limit to the specified entries for testing
    filtered_metadata = dict(islice(filtered_metadata.items(), 2))
    print(f"Using the first {len(filtered_metadata)} entries for testing.")

    verify_metadata(filtered_metadata)

    print("Creating dataset...")
    dataset = preprocess_pipeline(metadata)

    # Print out some example data from the dataset
#for data, label in dataset.take(3):  # Show first 3 samples
 #   print(f"Attributes: {data[:4]}")  # First 4 elements are the pet's attributes
  #  print(f"Image1: {data[4]}")  # Image 1
   # print(f"Image2: {data[5]}")  # Image 2
    #print(f"Label: {label}")  # Label: 1 or 0

    #print("Visualizing dataset samples...")
    #visualize_dataset_sample(dataset, num_samples=3)

TODO: label encoding, image augmentation?, performance optimization - parallel processing ...